# Calibrate Cameras

0.075 m
4x4 aruco a 1/8 bzw 6/8 * 0.075 

In [29]:
class Args:
    dataset_path = "../../resources/images/uncalibrated/calibration_dataset.npz"
    marker_length = 5.625
    square_length = 7.5

args = Args()

In [30]:
import numpy as np
import cv2
from cv2 import aruco
from matplotlib import pyplot as plt

from models.dataset.DatasetHelper import save_coefficients

In [1]:
def calibrate_charuco(images: list):
    '''Apply camera calibration using aruco.
    The dimensions are in cm.
    '''
    aruco_dict = aruco.Dictionary_get(aruco.DICT_4X4_50) 
    board = aruco.CharucoBoard_create(7, 5, args.square_length, args.marker_length, aruco_dict)
    arucoParams = aruco.DetectorParameters_create()

    image_id = -1
    image_ids, corners_list, id_list = [], [], []
    # Find the ArUco markers inside each image
    for image in images:
        image_id = image_id + 1

        img_gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        corners, ids, _ = aruco.detectMarkers(
            img_gray, 
            aruco_dict, 
            parameters=arucoParams
        )

        if ids is None: # did not detect any corners
            continue

        resp, charuco_corners, charuco_ids = aruco.interpolateCornersCharuco(
            markerCorners=corners,
            markerIds=ids,
            image=img_gray,
            board=board
        )

        if charuco_corners is None:
            continue
        
        # If a Charuco board was found, let's collect image/corner points
        # Requiring at least 20 squares
        if resp > 10:
            # Add these corners and ids to our calibration arrays
            corners_list.append(charuco_corners)
            id_list.append(charuco_ids)
            image_ids.append(image_id)

        # print markers on image
        aruco.drawDetectedMarkers(image, corners, ids)
        aruco.drawDetectedCornersCharuco(image, charuco_corners, charuco_ids)


    # Actual calibration
    _, cm, diffs, rvecs, tvecs = aruco.calibrateCameraCharuco(
        charucoCorners=corners_list, 
        charucoIds=id_list, 
        board=board, 
        imageSize=img_gray.shape, 
        cameraMatrix=None, 
        distCoeffs=None)
    
    print(cm)
    print(diffs)

    return rvecs, tvecs, image_ids

In [32]:
def get_homogeneous(rvecs, tvecs):
    hom = np.identity(4)
    rmat, _ = cv2.Rodrigues(np.reshape(rvecs, -1))
    hom[0:3, 0:3] = rmat
    hom[0:3, 3] = np.squeeze(tvecs[0])
    return hom

In [33]:
def inverse_affine(mat):
    rot = mat[0:3, 0:3]
    trans = mat[0:3, 3]
    rot_inv = np.linalg.inv(rot)
    trans_inv = -rot_inv @ trans
    inv_mat = np.identity(4)
    inv_mat[0:3, 0:3] = rot_inv
    inv_mat[0:3, 3] = trans_inv
    return inv_mat

### Calculate & save intr. & extr. matrix

In [34]:
from matplotlib import pyplot as plt
from models.dataset.DatasetHelper import openImageDataset

# load images
realsenseColor, _, zividColor, _ = openImageDataset(args.dataset_path)

# Calibrate 
rvecs_rs, tvecs_rs, image_ids_rs = calibrate_charuco(realsenseColor)
rvecs_ziv, tvecs_ziv, image_ids_ziv = calibrate_charuco(zividColor)

# Calculate both homogenous transformation matrixes
Ts_rs = []
Ts_ziv = []
ids = [(i, j)  for i, x in enumerate(image_ids_rs) for j, y in enumerate(image_ids_ziv) if x == y]

for idx, (id_rs, id_ziv) in enumerate(ids):
    Ts_rs.append(get_homogeneous(rvecs_rs[id_rs], tvecs_rs[id_rs]))
    Ts_ziv.append(get_homogeneous(rvecs_ziv[id_ziv], tvecs_ziv[id_ziv]))

Ts_ziv_to_rs = []
for T_rs, T_ziv in zip(Ts_rs, Ts_ziv):
    ex = inverse_affine(T_rs) @ T_ziv
    Ts_ziv_to_rs.append(ex)
    #print(ex)

T_ziv_to_rs = np.mean(Ts_ziv_to_rs, axis=0)
Ts_ziv_to_rs = Ts_ziv_to_rs[0]

print("Mean Transformation Matrix:")
print(T_ziv_to_rs)

print("Standard Deviation of Transformation Matrizes:")
print(np.std(Ts_ziv_to_rs, axis=0))

# plot images
f, axarr = plt.subplots(len(ids), 2, figsize=(10, 20))
for idx, img_id in enumerate(list(set(image_ids_rs) & set(image_ids_ziv))):
    axarr[idx][0].imshow(realsenseColor[img_id])
    axarr[idx][1].imshow(zividColor[img_id])

# Save transformation matrix into a file
# save_coefficients(T_ziv_to_rs, '../../resources/calibrations/calibration.yml')

Mean Transformation Matrix:
[[ 9.85694168e-01  1.62607245e-02 -1.54242802e-01  3.25331442e+01]
 [-1.66598873e-02  9.97242500e-01 -2.26440930e-03 -4.13724632e+01]
 [ 1.54579174e-01  4.40083588e-03  9.87294301e-01  3.37779913e+01]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]
Standard Deviation of Transformation Matrizes:
[ 0.40798734  0.43069049  0.45885949 32.65662036]
